In [81]:
# -------------- Imports und Einstellungen --------------
# Imports
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns

# Einstellungen
pd.set_option('display.max_colwidth', None)

# Ideen für Feature engineering: Durch prim_disease_hct	lässt sich vielleicht ein besseren füllwert für donor_age ermitteln. 

In [82]:
# -------------- Daten einlesen --------------

# Train Daten zum trainieren der ML-Modelle
path="/datasets/equity-post-HCT-survival-predictions/train.csv"
df=pd.read_csv(path)

# Spaltenbeschreibungen
path_description="/datasets/equity-post-HCT-survival-predictions/data_dictionary.csv"
df_description=pd.read_csv(path_description)

# Test Daten für die Competition
path_test="/datasets/equity-post-HCT-survival-predictions/test.csv"
df_test=pd.read_csv(path_test)

# # -------------- ID-Werte aus df_test speichern --------------

ids = df_test['ID']



In [25]:
# ------------------------------------------------------------- Datacleaning ------------------------------------------------------------- 



In [58]:
df.head()

,ID,dri_score,psych_disturb,cyto_score,diabetes,hla_match_c_high,hla_high_res_8,tbi_status,arrhythmia,hla_low_res_6,...,tce_div_match,donor_related,melphalan_dose,hla_low_res_8,cardiac,hla_match_drb1_high,pulm_moderate,hla_low_res_10,efs,efs_time
0,0,N/A - non-malignant indication,No,NaN,No,NaN,NaN,No TBI,No,6.0,...,NaN,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,42.356
1,1,Intermediate,No,Intermediate,No,2.0,8.0,"TBI +- Other, >cGy",No,6.0,...,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,Yes,10.0,1.0,4.672
2,2,N/A - non-malignant indication,No,NaN,No,2.0,8.0,No TBI,No,6.0,...,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,19.793
3,3,High,No,Intermediate,No,2.0,8.0,No TBI,No,6.0,...,Permissive mismatched,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,102.349
4,4,High,No,NaN,No,2.0,8.0,No TBI,No,6.0,...,Permissive mismatched,Related,MEL,8.0,No,2.0,No,10.0,0.0,16.223


In [83]:
# -------------- Zeilen mit vielen NaN-Werten bestimmen  -------------- 

# Verschiedene Schwellenwerte für NaN-Anteile (z. B. 10%, 30%, 50%, 70%)
thresholds = [0.2, 0.3,0.35,0.4,0.45, 0.5,0.64]  # Anteil an fehlenden Werten

# Überprüfung der Zeilen, die über jedem Schwellenwert liegen
for t in thresholds:
    threshold_value = t * df.shape[1]  # Berechne absolute Anzahl fehlender Werte
    count = (df.isna().sum(axis=1) > threshold_value).sum()
    print(f"Schwellenwert {int(t * 100)}%: {count} Zeilen haben mehr als {int(t * 100)}% NaN-Werte")



Schwellenwert 20%: 5200 Zeilen haben mehr als 20% NaN-Werte
Schwellenwert 30%: 2803 Zeilen haben mehr als 30% NaN-Werte
Schwellenwert 35%: 1212 Zeilen haben mehr als 35% NaN-Werte
Schwellenwert 40%: 409 Zeilen haben mehr als 40% NaN-Werte
Schwellenwert 45%: 220 Zeilen haben mehr als 45% NaN-Werte
Schwellenwert 50%: 149 Zeilen haben mehr als 50% NaN-Werte
Schwellenwert 64%: 2 Zeilen haben mehr als 64% NaN-Werte


In [84]:
# -------------- Zeilen mit vielen NaN-Werten löschen  -------------- 


threshold_value = 0.5 * df.shape[1]# Berechne absolute Anzahl fehlender Werte / Wert kann angepasst werden um zu überprüfen, ob geringere Werte besser sind für die ML-Modelle

before_cleaning = df.shape[0]

df = df[df.isna().sum(axis=1) <= threshold_value]

after_cleaning = df.shape[0]

# Ausgabe der Anzahl der gelöschten Zeilen
print(f"Anzahl der gelöschten Zeilen: {before_cleaning - after_cleaning}")

Anzahl der gelöschten Zeilen: 149


In [85]:
# -------------- NaN-Werte auffüllen für kategorische Spalten --------------

# Option 1: NaN-Werte mit dem am häufigsten vorkommenden Platzhalter ersetzen
placeholder_values = ['Not done', 'Not tested', 'TBD', 'N/A, F(pre-TED) not submitted', 'N/A, Mel not given']

# Kopie des DataFrames, um ungewollte Kopien zu vermeiden
df = df.copy()

# Iteriere über alle kategorischen Spalten
for col in df.select_dtypes(include=['object']).columns:
    # Filtere die Platzhalterwerte in der Spalte
    placeholders_in_col = df[col][df[col].isin(placeholder_values)]
    
    if not placeholders_in_col.empty:
        # Finde den häufigsten Platzhalterwert in der Spalte
        most_frequent_placeholder = placeholders_in_col.mode()[0]
        # Ersetze NaN-Werte mit dem häufigsten Platzhalterwert
        df[col] = df[col].fillna(most_frequent_placeholder)
    else:
        # Wenn keine Platzhalterwerte in der Spalte vorkommen, fülle NaN-Werte mit 'Unknown'
        df[col] = df[col].fillna('Unknown')



In [65]:
# -------------- NaN-Werte auffüllen für Kategorische spalten --------------

# Option 2: Alle NaN-Werte durch "Unknown" ersetzen

# Iteriere über alle numerischen Spalten
for col in df.select_dtypes(include=['number']).columns:
    # Fülle NaN-Werte in numerischen Spalten mit 'Unknown'
    df[col].fillna('Unknown', inplace=True)

In [86]:
# -------------- NaN-Werte auffüllen für Numerische spalten --------------

# Option 1: NaN-Werte mit dem Median ersetzen

# Iteriere über alle numerischen Spalten
for col in df.select_dtypes(include=['number']).columns:
    # Berechne den Median der Spalte und fülle NaN-Werte mit diesem Median
    median_value = df[col].median()
    df[col] = df[col].fillna(median_value)


In [87]:
# Überprüfe, ob noch NaN-Werte im DataFrame vorhanden sind
nan_check = df.isna().sum().sum()

if nan_check == 0:
    print("Es sind keine NaN-Werte mehr im DataFrame.")
else:
    print(f"Es gibt noch {nan_check} NaN-Werte im DataFrame.")


Es sind keine NaN-Werte mehr im DataFrame.


In [88]:
# -------------- ID-Spalte entfernen --------------

#ID-Spalte entfernen
df = df.drop(columns=['ID'])


In [89]:
# -------------- Kategorische Spalten One-Hot Encoden --------------

# Wende One-Hot-Encoding auf alle kategorialen Spalten an
df = pd.get_dummies(df, drop_first=False)

df.head()


,hla_match_c_high,hla_high_res_8,hla_low_res_6,hla_high_res_6,hla_high_res_10,hla_match_dqb1_high,hla_nmdp_6,hla_match_c_low,hla_match_drb1_low,hla_match_dqb1_low,...,donor_related_Unknown,donor_related_Unrelated,melphalan_dose_MEL,"melphalan_dose_N/A, Mel not given",cardiac_No,cardiac_Not done,cardiac_Yes,pulm_moderate_No,pulm_moderate_Not done,pulm_moderate_Yes
0,2.0,8.0,6.0,6.0,10.0,2.0,6.0,2.0,2.0,2.0,...,0,1,0,1,1,0,0,1,0,0
1,2.0,8.0,6.0,6.0,10.0,2.0,6.0,2.0,2.0,2.0,...,0,0,0,1,1,0,0,0,0,1
2,2.0,8.0,6.0,6.0,10.0,2.0,6.0,2.0,2.0,2.0,...,0,0,0,1,1,0,0,1,0,0
3,2.0,8.0,6.0,6.0,10.0,2.0,6.0,2.0,2.0,2.0,...,0,1,0,1,1,0,0,1,0,0
4,2.0,8.0,6.0,6.0,10.0,2.0,5.0,2.0,2.0,2.0,...,0,0,1,0,1,0,0,1,0,0


In [90]:
# # -------------- ID-Werte aus df_test speichern --------------

ids = df_test['ID']


In [91]:
# -------------- NaN-Werte auffüllen für Kategorische Spalten in df_test --------------

placeholder_values = ['Not done', 'Not tested', 'TBD', 'N/A, F(pre-TED) not submitted', 'N/A, Mel not given']

# Iteriere über alle kategorischen Spalten in df_test
for col in df_test.select_dtypes(include=['object']).columns:
    # Filtere die Platzhalterwerte in der Spalte
    placeholders_in_col = df_test.loc[df_test[col].isin(placeholder_values), col]
    
    if not placeholders_in_col.empty:
        # Finde den häufigsten Platzhalterwert in der Spalte
        most_frequent_placeholder = placeholders_in_col.mode()[0]
        # Fülle NaN-Werte mit dem häufigsten Platzhalterwert
        df_test[col] = df_test[col].fillna(most_frequent_placeholder)
    else:
        # Wenn keine Platzhalterwerte in der Spalte vorkommen, fülle NaN-Werte mit 'Unknown'
        df_test[col] = df_test[col].fillna('Unknown')

# -------------- NaN-Werte auffüllen für Numerische Spalten in df_test --------------

# Iteriere über alle numerischen Spalten in df_test und fülle NaN-Werte mit dem Median
df_test = df_test.fillna(df_test.median(numeric_only=True))

# -------------- Kategorische Spalten One-Hot Encoden --------------
df_test = pd.get_dummies(df_test, drop_first=False)

In [92]:
# Sicherstellen, dass df_test genau die gleichen Spalten wie df hat (außer 'efs')
df_test = df_test.reindex(columns=df.columns.drop('efs', errors='ignore'), fill_value=0)

# Falls df_test zusätzliche Spalten hat, die df nicht hat, diese entfernen
df_test = df_test[df.columns.drop('efs', errors='ignore')]


df_test.shape



(3, 197)

In [50]:
df_test.head()

,hla_match_c_high,hla_high_res_8,hla_low_res_6,hla_high_res_6,hla_high_res_10,hla_match_dqb1_high,hla_nmdp_6,hla_match_c_low,hla_match_drb1_low,hla_match_dqb1_low,...,donor_related_Unknown,donor_related_Unrelated,melphalan_dose_MEL,"melphalan_dose_N/A, Mel not given",cardiac_No,cardiac_Not done,cardiac_Yes,pulm_moderate_No,pulm_moderate_Not done,pulm_moderate_Yes
0,2.0,8.0,6.0,6.0,10.0,2.0,6.0,2.0,2.0,2.0,...,0,1,0,1,1,0,0,1,0,0
1,2.0,8.0,6.0,6.0,10.0,2.0,6.0,2.0,2.0,2.0,...,0,0,0,1,1,0,0,0,0,1
2,2.0,8.0,6.0,6.0,10.0,2.0,6.0,2.0,2.0,2.0,...,0,0,0,1,1,0,0,1,0,0


In [95]:
# -------------- KNN --------------

# 1. Features und Zielvariable definieren
X = df.drop(columns=['efs'])  # Features aus df
y = df['efs']  # Zielvariable EFS

# 2. Trainingsdaten skalieren
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 3. KNN-Modell mit k=5 trainieren
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_scaled, y)

# 4. Testdaten vorbereiten (ohne efs)
X_test = df_test.drop(columns=['efs'], errors='ignore')  # Falls efs fälschlicherweise drin ist
X_test_scaled = scaler.transform(X_test)  # Gleiche Skalierung anwenden

# 5. Vorhersagen für df_test
efs_predictions = knn.predict(X_test_scaled)

# 6. IDs und Vorhersagen in einem neuen DataFrame speichern
df_pred = pd.DataFrame({
    'ID': ids,  # IDs aus df_test
    'result': efs_predictions  # Vorhersagen in der Spalte 'result'
})

# 7. DataFrame als CSV speichern
df_pred.to_csv('predictions.csv', index=False)

# 8. Ausgabe der ersten Zeilen
print(df_pred.head())  # Ausgabe der ersten Zeilen von df_pred

      ID  result
0  28800     0.0
1  28801     1.0
2  28802     0.0


In [96]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import make_scorer, roc_auc_score

# Neue Bewertungsmetrik: ROC AUC
roc_auc_scorer = make_scorer(roc_auc_score, multi_class='ovr')

# 1. Features und Zielvariable definieren
X = df.drop(columns=['efs'])  # Features
y = df['efs']  # Zielvariable

# 2. Random Forest-Modell definieren
rf = RandomForestClassifier(random_state=42, n_jobs=-1)

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2'],
    'bootstrap': [True, False]
}

# Fortschrittsanzeige
print("🔍 Starte GridSearchCV für Random Forest...")
print(f"🔄 Teste {np.prod([len(v) for v in param_grid.values()])} verschiedene Hyperparameter-Kombinationen...")

# 4. GridSearchCV initialisieren
grid_search = GridSearchCV(
    estimator=rf, 
    param_grid=param_grid,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),  
    n_jobs=-1, 
    verbose=1,
    scoring='accuracy'
)

# 5. GridSearchCV ausführen
print("🚀 Training gestartet...")
grid_search.fit(X, y)  # Kein X_scaled mehr!

# 6. Beste Parameter ausgeben
print("\n✅ GridSearch abgeschlossen!")
print("Beste Hyperparameter:", grid_search.best_params_)
print("Beste Genauigkeit:", grid_search.best_score_)

# 7. Optimiertes Modell speichern
best_rf = grid_search.best_estimator_
print("\n💾 Optimiertes Modell gespeichert als 'best_rf'")



🔍 Starte GridSearchCV für Random Forest...
🔄 Teste 648 verschiedene Hyperparameter-Kombinationen...
🚀 Training gestartet...
Fitting 5 folds for each of 648 candidates, totalling 3240 fits
[CV 2/5] END max_depth=10, min_samples_leaf=3, min_samples_split=2, n_estimators=200;, score=0.980 total time=  15.4s
[CV 5/5] END max_depth=15, min_samples_leaf=3, min_samples_split=2, n_estimators=200;, score=0.980 total time=  26.1s
[CV 5/5] END max_depth=20, min_samples_leaf=5, min_samples_split=2, n_estimators=200;, score=0.980 total time=  26.8s
[CV 5/5] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, score=0.979 total time=   2.1s
[CV 3/5] END bootstrap=True, max_depth=10, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=300;, score=0.971 total time=  14.8s
[CV 3/5] END bootstrap=True, max_depth=20, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=300;, score=0.984 total time=  24.9s

/opt/conda/envs/survival-env/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV 3/5] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, score=0.983 total time=   5.7s
[CV 3/5] END max_depth=15, min_samples_leaf=1, min_samples_split=10, n_estimators=100;, score=0.984 total time=  17.5s
[CV 1/5] END max_depth=20, min_samples_leaf=1, min_samples_split=2, n_estimators=300;, score=0.982 total time=  20.6s
[CV 2/5] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=300;, score=nan total time=   0.0s
[CV 3/5] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=200;, score=0.983 total time=  16.6s
[CV 1/5] END bootstrap=True, max_depth=20, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200;, score=nan total time=   0.3s
[CV 2/5] END bootstrap=True, max_depth=20, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200;, score=nan total time=   0.2s
[CV 4/5] END bootstrap=True, max_depth=

/opt/conda/envs/survival-env/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:547: FitFailedWarning: 
1080 fits failed out of a total of 3240.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
548 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/conda/envs/survival-env/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/envs/survival-env/lib/python3.9/site-packages/sklearn/base.py", line 1467, in wrapper
    estimator._validate_params()
  File "/opt/conda/envs/survival-env/lib/python3.9/site-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_con


✅ GridSearch abgeschlossen!
Beste Hyperparameter: {'bootstrap': False, 'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
Beste Genauigkeit: 0.9831768499477596

💾 Optimiertes Modell gespeichert als 'best_rf'


In [97]:
# -------------- Random Forest --------------

# 1. Features und Zielvariable definieren
X = df.drop(columns=['efs'])  # Features
y = df['efs']  # Zielvariable

# 2. Random Forest Modell trainieren (mit besten Parametern aus GridSearch)
rf = best_rf  # <- Das beste Modell aus GridSearch verwenden!
rf.fit(X, y)

# 3. Testdaten vorbereiten (ohne 'efs')
X_test = df_test.drop(columns=['efs'], errors='ignore')  # Falls efs fälschlicherweise drin ist

# 4. Vorhersagen für df_test
efs_predictions = rf.predict(X_test)

# 5. IDs und Vorhersagen in einem neuen DataFrame speichern
df_pred = pd.DataFrame({
    'ID': ids,  # IDs aus df_test
    'prediction': efs_predictions  # Vorhersagen in der Spalte 'result'
})

# 6. DataFrame als CSV speichern
df_pred.to_csv('submission.csv', index=False)

# 7. Ausgabe der ersten Zeilen
print(df_pred.head())  # Ausgabe der ersten Zeilen von df_pred


      ID  prediction
0  28800         1.0
1  28801         1.0
2  28802         1.0


In [77]:
df.shape

(28651, 198)

In [80]:
print(df['efs'].value_counts(normalize=True))

1.0    0.539306
0.0    0.460694
Name: efs, dtype: float64
[CV 5/5] END max_depth=None, min_samples_leaf=2, min_samples_split=2, n_estimators=100;, score=0.982 total time=   2.1s
[CV 3/5] END max_depth=None, min_samples_leaf=2, min_samples_split=5, n_estimators=300;, score=0.987 total time=   6.5s
[CV 1/5] END max_depth=None, min_samples_leaf=4, min_samples_split=2, n_estimators=300;, score=0.980 total time=   8.0s
[CV 1/5] END max_depth=None, min_samples_leaf=2, min_samples_split=5, n_estimators=100;, score=0.981 total time=   3.1s
[CV 5/5] END max_depth=None, min_samples_leaf=2, min_samples_split=10, n_estimators=200;, score=0.981 total time=   4.3s
[CV 2/5] END max_depth=None, min_samples_leaf=4, min_samples_split=5, n_estimators=200;, score=0.979 total time=   7.1s
[CV 2/5] END max_depth=None, min_samples_leaf=2, min_samples_split=2, n_estimators=300;, score=0.979 total time=   5.2s
[CV 1/5] END max_depth=None, min_samples_leaf=4, min_samples_split=2, n_estimators=100;, score=0.980 